# 04. 영어/한국어 Word2Vec 훈련시키기
---
gensim 패키지에서 Word2Vec은 이미 구현되어져 있으므로 별도의 word2vec을 구현할 필요 없이 손쉽게 훈련 가능

In [4]:
# 1. 영어 Word2Vec 만들기
# 영어 데이터를 다운 받아 직접 word2vec 작업을 진행

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lemon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
# 훈련 데이터
# 링크 : https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip
# 위 파일 압축을 풀면 ted_en-2016408.xml 파일을 얻을 수 있음

datapath = 'D:/chch/startwhithtorch/datafolder/'
urllib.request.urlretrieve('https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml',
                           filename = datapath + 'ted_en-2016408.xml')

('D:/chch/startwhithtorch/datafolder/ted_en-2016408.xml',
 <http.client.HTTPMessage at 0x256b2ab1b80>)

In [7]:
# 해당 파일은 .xml로 우리가 원하는 자연어만 얻기 위해서는 전처리가 필요하다.
# 얻고자 하는 실질적인 데이터는 영어문장으로만 구성된 내용을 담고 있는
# <content> 와 </content> 사이의 내용이다.
# 추가로 사이의 내용에서 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 제거해야 한다.

### xml 파일 예시

<file id="1">  
  <head>  
<url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them</url>  
       <pagesize>72832</pagesize>  
... xml 문법 중략 ...  
<content>  
Here are two reasons companies fail: they only do more of the same, or they only do what's new.  
To me the real, real solution to quality growth is figuring out the balance between two activities:  
... content 내용 중략 ...  
To me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap,   small electronic calculators in Japan that they used to double-check their calculators.   
(Laughter)  
... content 내용 중략 ...  
(Applause)  
</content>  
</file>  
<file id="2">  
    <head>  
<url>http://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space<url>  
... 이하 중략 ...  

In [8]:
# 훈련 데이터 전처리하기
targetXML = open(datapath + 'ted_en-2016408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '/n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'/([^)]*/)', '', parse_text)
# parse_text 를 불러와서 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거한다는 뜻

In [9]:
len(content_text)

23430418

In [10]:
sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용해 문장 토큰화(sentence tokenize) 진행

In [11]:
sent_text[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.',
 'Both are necessary, but it can be too much of a good thing.',
 'Consider Facit.',
 "I'm actually old enough to remember them."]

In [12]:
normalized_text = []
for string in sent_text :
    tokens = re.sub(r'[^a-z0-9]+', ' ', string.lower())
    normalized_text.append(tokens)
# 각 문장에서 알파벳소문자와 숫자를 제외하고는 블랭크로 바꿈

In [13]:
normalized_text[:5]

['here are two reasons companies fail they only do more of the same or they only do what s new ',
 'to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation ',
 'both are necessary but it can be too much of a good thing ',
 'consider facit ',
 'i m actually old enough to remember them ']

In [14]:
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용해 단어 토큰화 수행

In [15]:
result[:5]

[['here',
  'are',
  'two',
  'reasons',
  'companies',
  'fail',
  'they',
  'only',
  'do',
  'more',
  'of',
  'the',
  'same',
  'or',
  'they',
  'only',
  'do',
  'what',
  's',
  'new'],
 ['to',
  'me',
  'the',
  'real',
  'real',
  'solution',
  'to',
  'quality',
  'growth',
  'is',
  'figuring',
  'out',
  'the',
  'balance',
  'between',
  'two',
  'activities',
  'exploration',
  'and',
  'exploitation'],
 ['both',
  'are',
  'necessary',
  'but',
  'it',
  'can',
  'be',
  'too',
  'much',
  'of',
  'a',
  'good',
  'thing'],
 ['consider', 'facit'],
 ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']]

In [16]:
print(f'총 샘플의 개수: {len(result)}')

총 샘플의 개수: 265335


In [18]:
# 3) Word2Vec 훈련시키기
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(
                 sentences = result,    # 훈련시킬 문장셋
                                        # 각 문장마다 하나의 토큰 list를 생성하며 토큰 list의 개수는 문장 개수 n개 만큼 생성하여 sentences에 저장함
                 vector_size = 100,     # 워드 벡터의 특징 값, 워드 벡터의 차원
                 window = 5,            # 현재 단어와 예측 단어의 최대 거리
                 min_count = 5,         # 이 수보다 낮은 빈도수의 단어는 무시
                 workers = 4,           # 모델 생성시 사용할 쓰레드 개수
                 sg = 0                 # 1 = skip-gram / 0 = CBOW
                 )

In [19]:
# 학습을 했으니 이용해보자

# .wv.most_similar()로 가장 유사한 단어들을 출력해보자
model_result = model.wv.most_similar('woman')
print(model_result)

[('man', 0.8601292967796326), ('girl', 0.8386016488075256), ('lady', 0.8307408094406128), ('boy', 0.7759738564491272), ('soldier', 0.7337160110473633), ('child', 0.7305009365081787), ('kid', 0.7267239093780518), ('gentleman', 0.7170665264129639), ('doctor', 0.7078558802604675), ('guy', 0.7008314728736877)]


In [20]:
# 4) Word2Vec 모델 저장하고 로드하기
# 학습한 모델을 언제든 다시 사용할 수 있도록 저장해보자

# 모델 저장
model.wv.save_word2vec_format('./eng_w2v')
# 모델 로드
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [21]:
# 저장한 모델 불러와서 유사한 단어 뽑아보기
model_result = loaded_model.most_similar('sky')
print(model_result)

[('sun', 0.7738822102546692), ('river', 0.7321430444717407), ('sea', 0.7295847535133362), ('ocean', 0.7232835292816162), ('mountain', 0.7224076390266418), ('sand', 0.721751868724823), ('moon', 0.7213958501815796), ('desert', 0.7180595993995667), ('window', 0.7092677354812622), ('lake', 0.7088762521743774)]


## 2. 한국어 Word2Vec 만들기
---

In [ ]:
# 위키피티아 한국어 덤프 파일을 다운 받아서 한국어로 w2v을 진행해보자

In [22]:
# 1) 위키피디아 한국어 덤프 파일 다운로드
# 링크: https://dumps.wikimedia.org/kowiki/latest/
# 사용할 파일: kowiki-latest-pages-articles.xml.bz2

# >>> wikiko 다운 받았음

In [23]:
# 2) 위키피디아 익스트랙터 다운로드

# >>> wikiko 다운 받았음

In [ ]:
# 3) 위키피디아 한국어 덤프 파일 변환
# 위 두개의 파일들을 같은 폴더에 위치시킨 후에 아래 명령어를 실행
# python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 

# 안 되서 갸~빡침